In [1]:
import numpy as np
import mocalum as mc
from mocalum import plot_mocalum_setup, spher2cart

In [ ]:
koshava_xyz = [500, 500, 0]
no_scans = 1000
# wind field setup
ref_height = 100      # power law reference height
meas_height = 100 
shear_exponent = 0.2  # power law shear exponent
wind_speed = 8       # wind speed at referenec height
wind_dir = 23         # wind direction
w = 0 

# beam steering setup
distance = 1000   # meter
elevation = np.degrees(np.arcsin(meas_height / distance))
angular_res = 1   # degree
azimuth_mid = 45  # central azimuth angle
sector_size = 30  # degree
scan_speed = 1    # degree.s^-1
max_speed = 50    # degree.s^-1
max_acc = 100     # degree.s^-2

# Uncertainty terms
no_sim = 10000 # number of simulations
corr_coef = 0  # correlation coefficient
mu = 0         # we assume no systematic uncertainty
azim_std = 0.1 # degrees
elev_std = 0.1 # degrees
dis_std = 10   # meters
rad_std = 0.1  # m.s-1  In [23]:  

tmp = mc.Mocalum()
# tmp.x_res = tmp.y_res = 10
  
tmp.set_ivap_probing(koshava_xyz, sector_size, azimuth_mid, angular_res, elevation, distance, 
                    no_scans, scan_speed, max_speed,max_acc)

tmp.gen_unc_contributors(corr_coef)


tmp.x_res = tmp.y_res = 25
tmp.z_res = 5
tmp.turbbox_time = 100

tmp.gen_turb_ffield(wind_speed, wind_dir, w, ref_height, shear_exponent)
plot_mocalum_setup(tmp)

In [11]:
# 4D tbox
tmp.data.ffield

<xarray.Dataset>
Dimensions:  (time: 21, x: 13, y: 25, z: 7)
Coordinates:
  * time     (time) float64 0.0 3.125 6.25 9.375 12.5 ... 53.12 56.25 59.38 62.5
  * y        (y) float64 -927.7 -902.7 -877.7 -852.7 ... -377.7 -352.7 -327.7
  * z        (z) float64 87.45 92.45 97.45 102.4 107.4 112.4 117.4
  * x        (x) float64 1.416e+03 1.441e+03 1.466e+03 ... 1.691e+03 1.716e+03
Data variables:
    u        (time, z, y, x) float64 7.151 6.331 7.556 10.92 ... 8.3 7.459 7.385
    v        (time, z, y, x) float64 -0.1528 1.347 -0.2941 ... 0.6945 1.27 1.019
    w        (time, z, y, x) float64 -0.4372 0.7017 -0.05603 ... -1.694 0.009228
Attributes:
    generator:  PyConTurb
    title:      Turbulent flow field dataset

In [3]:
# 3D tbox, it is hiddedn but still there as a private variable
tmp.data._ffield

<xarray.Dataset>
Dimensions:  (time: 33, y: 25, z: 7)
Coordinates:
  * time     (time) float64 0.0 3.125 6.25 9.375 ... 90.62 93.75 96.88 100.0
  * y        (y) float64 -924.4 -899.4 -874.4 -849.4 ... -374.4 -349.4 -324.4
  * z        (z) float64 86.64 91.64 96.64 101.6 106.6 111.6 116.6
Data variables:
    u        (z, y, time) float64 4.396 4.241 6.537 7.159 ... 6.066 5.814 6.003
    v        (z, y, time) float64 -2.01 -1.683 -0.4037 ... -0.2371 -1.874 0.556
    w        (z, y, time) float64 0.71 1.793 1.524 ... 0.3404 1.391 -0.2473
Attributes:
    title:      Turbulent flow field dataset
    generator:  PyConTurb